In [6]:
import numpy as np
import os                                             #pode vir a ser útil quando tivermos mais arquivos
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from calendar import month_name
!pip install --upgrade "ibm-watson>=5.0.0"
from ibm_watson import LanguageTranslatorV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import sys

     |████████████████████████████████| 382 kB 17.7 MB/s 
     |████████████████████████████████| 198 kB 33.1 MB/s 
  Created wheel for ibm-watson: filename=ibm_watson-5.1.0-py3-none-any.whl size=375438 sha256=2068b881641b2dee7076007fea1b96f52660a6b333ef48e901aed48307fdb1e8
  Stored in directory: /home/datalore/.cache/pip/wheels/c1/40/da/ba44de639af32759a5755cb79afedf309104b13fea09bbb5fc
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-3.5.1-py3-none-any.whl size=56746 sha256=56cc4070d4cb5a5388131679b0942bc52e7b0ea02235303bc7a8d2110f1fb521
  Stored in directory: /home/datalore/.cache/pip/wheels/2b/82/dd/1535dd033203c5e255eb9afc256619627300aa2b6405c14f04
Successfully built ibm-watson ibm-cloud-sdk-core


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/datalore/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Pegando a data base

In [7]:
df_fake = pd.read_csv('Fake.csv',encoding = 'UTF-8')
df_true = pd.read_csv('True.csv',encoding = 'UTF-8')

df_fake['reality'] = 0
df_true['reality'] = 1
df_text = df_true.append(df_fake, ignore_index = True)
df_text['text'] = df_text['title'].str.lower() + ' ' + df_text['text'].str.lower()
df_text.drop(columns = ['subject','date','title'], inplace = True)

for word in stopwords.words('english'):
  df_text['text'] = df_text['text'].str.replace(' ' + word + ' ',' ')

for word in month_name:
  df_text['text'] = df_text['text'].str.replace(' ' + word.lower() + ' ',' ')

# Separando as listas de textos e labels

In [8]:
textos_totais = df_text['text']  # lista do pandas - ver se funciona - video usa lista normal
labels = df_text['reality']

# Definindo o recorte de treino e de teste

In [9]:
'''
tamanho_do_treino = 5800
tamanho_do_teste = len(labels) - tamanho_do_treino

# definindo o corte do treino e teste
dados_treino = textos_totais[700:tamanho_do_treino+700]
dados_teste = textos_totais[0:700]
labels_treino = labels[0:tamanho_do_treino]
labels_teste = labels[tamanho_do_treino:]
'''

'\ntamanho_do_treino = 5800\ntamanho_do_teste = len(labels) - tamanho_do_treino\n\n# definindo o corte do treino e teste\ndados_treino = textos_totais[700:tamanho_do_treino+700]\ndados_teste = textos_totais[0:700]\nlabels_treino = labels[0:tamanho_do_treino]\nlabels_teste = labels[tamanho_do_treino:]\n'

In [10]:
dados_treino, dados_teste, labels_treino, labels_teste = train_test_split(textos_totais, labels, test_size=0.25)

In [11]:
print(textos_totais[5620])
print(labels[5620])
print(dados_teste)
print(labels_teste)

u.s. tech titans lead legal brief trump travel ban (reuters) - 100 companies, including high-tech’s biggest names, joined legal brief opposing president donald trump’s temporary travel ban, arguing would give companies strong incentives move jobs outside united states. companies - including apple inc (aapl.o), google inc (googl.o) microsoft corp (msft.o) - banded together late sunday file “friend-of-the-court” brief 9th u.s. circuit court appeals san francisco. argued executive order temporarily banning citizens seven muslim-majority countries refugees “inflicts significant harm american business.” elon musk’s tesla inc (tsla.o) spacex among companies signing onto brief monday, raising number signatories 127. musk among tech executives trump’s business advisory council. forced defend participation recent days, particularly since uber technologies inc’s [uber.ul] chief executive, travis kalanick, quit group thursday following travel ban. new republican president’s jan. 27 executive orde

# Tokenizando as palavras mais importantes de treino

In [44]:
token = Tokenizer(num_words = 2000, oov_token='<OOV>')  # n de palavras a serem tokenizadas - token <oov> para não encontradas
token.fit_on_texts(dados_treino)  # tokeniza as mais importantes dos dados de TREINO
indice_palavras = token.word_index  # cria um indice das palavras tokenizadas (tem forma de dict)
print(indice_palavras)

107, '2007in': 82108, 'newfangled': 82109, '2007president': 82110, '2008rep': 82111, '“anthony': 82112, '“huma': 82113, 'gege': 82114, 'wati': 82115, 'borunda': 82116, 'lamone': 82117, '2017frosh': 82118, "executive's": 82119, '“bent': 82120, 'shootout”': 82121, 'delegitimized': 82122, '“bridge': 82123, 'builder”': 82124, 'jousted': 82125, 'chitchat': 82126, 'shorelines': 82127, 'tylenols': 82128, 'equalize': 82129, 'ozdinec': 82130, 'overpaying': 82131, 'wtmaxine': 82132, 'stree': 82133, 'included”': 82134, '64vx7wkynj': 82135, '1010puppytoes': 82136, 'lazura': 82137, 'lanouaille': 82138, 'sunsetcliffs99': 82139, '2017hypocrisy': 82140, 'tag1113': 82141, 'theresagiudice': 82142, 'gunk': 82143, 'tgunkster': 82144, 'loblaw': 82145, 'mrgregshields': 82146, "80's": 82147, 'deedee33955': 82148, 'reneejacobslens': 82149, 'heilbronner': 82150, 'stimme': 82151, 'hilmer': 82152, 'militia’s': 82153, 'encima': 82154, 'irvine’s': 82155, 'dandimicco': 82156, 'clny': 82157, 'csg': 82158, 'clg': 821

In [46]:
x = 0
for i in indice_palavras:
    print(i)
    x += 1
    if x == 100:
        break

<OOV>
trump
said
u
s
president
would
people
”
one
state
new
also
reuters
house
donald
clinton
obama
government
states
republican
year
could
white
united
told
like
campaign
two
election
time
news
last
party
first
the
hillary
washington
former
even
us
country
video
media
years
many
american
it
security
law
police
say
political
national
made
get
court
make
right
since
twitter
russia
republicans
going
back
percent
presidential
bill
according
says
america
administration
vote
north
support
week
called
via
democratic
senate
trump’s
group
think
know
way
officials
including
office
public
take
military
well
world
foreign
federal
million
000
day
statement
tax


# Sequenciando tokens de treino

In [13]:
sequencias_treino = token.texts_to_sequences(dados_treino)  # organizadas os tokens de treino em sequencia numéricas
print(sequencias_treino)

544, 1, 342, 823, 1, 1, 1, 1, 293, 472, 152, 1, 837, 1139, 287, 2, 316, 1200, 242, 46, 21, 1, 444, 293, 3, 1, 2, 1033, 223, 404, 1453, 1, 442, 397, 586, 1559], [1156, 156, 1, 1, 1, 1, 1, 1, 1018, 1, 1, 6, 18, 269, 759, 9, 43, 604, 1, 6, 16, 2, 902, 1, 156, 611, 1, 641, 36, 641, 479, 98, 12, 6, 1, 902, 1759, 91, 347, 156, 1829, 1109, 130, 103, 343, 1, 1409, 32, 12, 1, 349, 23, 506, 481, 726, 489, 26, 1817, 256, 29, 1, 1130, 1049, 388, 1702, 19, 608, 23, 1, 117, 252, 347, 23, 168, 460, 282, 127, 3, 86, 392, 1, 204, 1049, 1, 941, 1, 349, 1270, 796, 1500, 1, 1, 812, 888, 26, 299, 24, 15, 256, 1430, 72, 1, 749, 156, 1, 49, 1, 1, 1427, 1199, 503, 546, 1, 47, 440, 946, 257, 1, 1350, 156, 1, 3, 2, 187, 1, 132, 1245, 91, 347, 366, 1160, 1535, 1075, 234, 1917, 1, 1895, 642, 233, 174, 1, 1, 260, 18, 1, 1, 13, 1, 130, 156, 1, 1, 642, 77, 1, 1, 1874, 174, 493, 1072, 201, 800, 1230, 1, 1, 201, 130, 1325, 1, 184, 489, 2, 742, 1717, 584, 1, 130, 611, 187, 249, 44, 250, 1, 1428, 1804, 2, 1, 1, 1089, 18

# Adicionando padding no final das frases para que todas tenha o mesmo tamanho (max = 300 palavras)

In [14]:
# o que fazer quando a frase tem tamanhos diferentes? avançada: RaggedTensor / simples: padding
# comando abaixo faz com que todas as frases tenham o mesmo tamanho da maior entre elas
com_padding_treino = pad_sequences(sequencias_treino, padding='post',  # podemos truncar palavras axima do max length
                            maxlen=300, truncating='post')  # podemos especificar o tamnho maximo das palavras espaçadas
print(com_padding_treino)  # post faz com que o padding fique no final

[[ 142  364    1 ...    0    0    0]
 [   4    5   20 ... 1978    1  463]
 [  79   34  155 ...    0    0    0]
 ...
 [1976   70    1 ...  199    1    1]
 [   1    1  905 ...    0    0    0]
 [ 709  206   41 ...    0    0    0]]


In [15]:
print(com_padding_treino[0])  # ver a primeira sequencia numerica
print(com_padding_treino.shape)  # ver quantas sequencias temos e de que tamanho são

[ 142  364    1    1 1010 1296    1   70    1    1   37    1    1    6
  485   56  814  365    9   39   15  648  281  511   15  145    1   19
  581    1    1    1  998    1  364    1  150   33  357  364    1  866
    1    1  676   37   17  872  325 1475    1   50  132    1   23  552
  396  132    1  895    1    1  364    1    1   22  267   11  101 1188
    1  516  713  140  723 1527  675  353  137  814  110   27    1    1
 1808    1  261    1    1  261  353  137    3    7    1   64 1724    1
    5 1078    1  152    7  155  192  146 1175   11  308 1667   65  144
  180  172  364    1  132  353  137 1355  814    1  269  318  928    1
    6  485   56  814  365  364    1   13  132    1 1355  814  421    1
    1   92   84  140  325  221   64  322    1    1  751  142    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

# Adicionando padding nas frases de teste para terem o mesmo formato das de treino

In [16]:
sequencias_teste = token.texts_to_sequences(dados_teste)  # organizadas os tokens de teste em sequencia numéricas
com_padding_teste = pad_sequences(sequencias_teste, padding='post',  # podemos truncar palavras axima do max length
                            maxlen=300, truncating='post')  # podemos especificar o tamnho maximo das palavras espaçadas
print(com_padding_teste)  # post faz com que o padding fique no final

[[  15  184  207 ...    0    0    0]
 [   1    1    1 ...    0    0    0]
 [  38    1  100 ...   72    7    1]
 ...
 [  18    1  503 ...    0    0    0]
 [1607 1725  570 ...    0    0    0]
 [1330  867   68 ...    0    0    0]]


# Tranformando as lista em np arrays (necessário para funcionar)

In [17]:
com_padding_treino = np.array(com_padding_treino)
labels_treino = np.array(labels_treino)
com_padding_teste = np.array(com_padding_teste)
labels_teste = np.array(labels_teste)

# Criando a rede neural

In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(2000, 16, input_length=300),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])  
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Algumas infos sobre o modelo

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 16)           32000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                408       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 32,433
Trainable params: 32,433
Non-trainable params: 0
_________________________________________________________________


# Definindo o número de repetições e rodando o modelo

In [20]:
num_epochs = 10
history = model.fit(com_padding_treino, labels_treino, epochs=num_epochs,
                    validation_data=(com_padding_teste, labels_teste), verbose=2)

Epoch 1/10
1053/1053 - 4s - loss: 0.2236 - accuracy: 0.9473 - val_loss: 0.0528 - val_accuracy: 0.9875
Epoch 2/10
1053/1053 - 4s - loss: 0.0323 - accuracy: 0.9936 - val_loss: 0.0279 - val_accuracy: 0.9929
Epoch 3/10
1053/1053 - 4s - loss: 0.0158 - accuracy: 0.9969 - val_loss: 0.0200 - val_accuracy: 0.9952
Epoch 4/10
1053/1053 - 4s - loss: 0.0092 - accuracy: 0.9983 - val_loss: 0.0163 - val_accuracy: 0.9955
Epoch 5/10
1053/1053 - 4s - loss: 0.0058 - accuracy: 0.9991 - val_loss: 0.0192 - val_accuracy: 0.9941
Epoch 6/10
1053/1053 - 4s - loss: 0.0036 - accuracy: 0.9994 - val_loss: 0.0147 - val_accuracy: 0.9960
Epoch 7/10
1053/1053 - 4s - loss: 0.0021 - accuracy: 0.9997 - val_loss: 0.0146 - val_accuracy: 0.9963
Epoch 8/10
1053/1053 - 4s - loss: 0.0013 - accuracy: 0.9998 - val_loss: 0.0150 - val_accuracy: 0.9968
Epoch 9/10
1053/1053 - 5s - loss: 7.5636e-04 - accuracy: 0.9999 - val_loss: 0.0161 - val_accuracy: 0.9966
Epoch 10/10
1053/1053 - 5s - loss: 5.1591e-04 - accuracy: 0.9999 - val_loss: 0

In [21]:
def ajustar_texto(texto):
  texto = texto.lower()
  texto = texto.replace('\n', ' ')
  texto = texto.strip()
  return texto

In [39]:
noticia = '''


The Senate approved a budget bill early Friday that paves the way for passage of President Biden’s $1.9 trillion coronavirus relief package, with Vice President Harris casting the tiebreaking vote on the measure that will be key to enacting Biden’s first major legislative initiative.

Passage of the budget bill by a 51-to-50 vote came just after 5:30 a.m. Friday, after an all-night Senate session during which senators plowed through dozens of amendments in a chaotic process known as a “vote-a-rama.” Democrats cheered on progress to address the pandemic, while Republicans complained of partisanship and excessive spending.



'''

noticia_ajustada = ajustar_texto(noticia)
print(noticia_ajustada)

the senate approved a budget bill early friday that paves the way for passage of president biden’s $1.9 trillion coronavirus relief package, with vice president harris casting the tiebreaking vote on the measure that will be key to enacting biden’s first major legislative initiative.  passage of the budget bill by a 51-to-50 vote came just after 5:30 a.m. friday, after an all-night senate session during which senators plowed through dozens of amendments in a chaotic process known as a “vote-a-rama.” democrats cheered on progress to address the pandemic, while republicans complained of partisanship and excessive spending.


In [40]:
noticia_na_lista = []
noticia_na_lista.append(noticia_ajustada)
print(noticia_na_lista)

['the senate approved a budget bill early friday that paves the way for passage of president biden’s $1.9 trillion coronavirus relief package, with vice president harris casting the tiebreaking vote on the measure that will be key to enacting biden’s first major legislative initiative.  passage of the budget bill by a 51-to-50 vote came just after 5:30 a.m. friday, after an all-night senate session during which senators plowed through dozens of amendments in a chaotic process known as a “vote-a-rama.” democrats cheered on progress to address the pandemic, while republicans complained of partisanship and excessive spending.']


In [41]:
sequencia = token.texts_to_sequences(noticia_na_lista)
sequencia = np.array(sequencia)
noticia_com_padding = pad_sequences(sequencia, maxlen=300, padding='post', truncating='post')
print(noticia_com_padding)

[[  36   80 1162  225  427   68  460  127  146    1   36   85  707    1
  1080    6    1  105  437    1    1 1959    1 1340  650    6    1    1
    36    1   73  412   36 1112  146    1 1171  647  485    1    1   35
   262 1504    1    1 1080   36  427   68 1821  225    1  485  794   73
   264    1 1583  219  452  225 1369  127 1583    1  459  357   80    1
     1    1  971    1    1 1655 1080    1  116  225    1  400  309  898
   225    1  225    1    9  121    1  412 1176  485  797   36    1 1533
    63    1 1080    1  300    1  639    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0   

In [42]:
previsao = model.predict(noticia_com_padding)
print(previsao)

[[1.9904077e-07]]


# Avaliando com base brasileira traduzida:

In [41]:
def translate(texts, source, target):
    key = 'f6W87dCMFbiJo1VTfW31BhRtf33ug2lgAjk0J6tQld9N'

    authenticator = IAMAuthenticator(key)
    language_translator = LanguageTranslatorV3(
        version='2018-05-01',
        authenticator=authenticator
    )

    language_translator.set_service_url('https://api.us-south.language-translator.watson.cloud.ibm.com')

    translation = language_translator.translate(
        text=texts,
        model_id='{1}-{0}'.format(target,source)).get_result()
    
    out = [item['translation'] for item in translation['translations']]
    if len(out) == 1: return out[0]
    return [item['translation'] for item in translation['translations']]

In [43]:
max_files = 25
source = 'pt'
destination = 'en'
texts = []

for file in os.listdir('./fake')[0:max_files]:
    texts.append(translate(open('./fake/' + file).read().lower(),source,destination))

for file in os.listdir('./true')[0:max_files]:
    texts.append(translate(open('./true/' + file).read().lower(),source,destination))

df_portugues = pd.DataFrame({'text':texts, 'reality':[0]*max_files + [1]*max_files})
for word in stopwords.words('english'):
  df_portugues['text'] = df_portugues['text'].str.replace(' ' + word + ' ',' ')

for word in month_name:
  df_portugues['text'] = df_portugues['text'].str.replace(' ' + word.lower() + ' ',' ')

print(df_portugues)

textos_totais = df_portugues['text']  # lista do pandas - ver se funciona - video usa lista normal
labels = df_portugues['reality']
sequencias_teste = token.texts_to_sequences(textos_totais)  # organizadas os tokens de teste em sequencia numéricas
com_padding_teste = pad_sequences(sequencias_teste, padding='post',  # podemos truncar palavras axima do max length
                            maxlen=300, truncating='post')  # podemos especificar o tamnho maximo das palavras espaçadas
com_padding_teste = np.array(com_padding_teste)
labels_teste = np.array(labels)
prediction = model.evaluate(com_padding_teste,labels_teste)
print(prediction)

                                                 text  reality
0   urgent: Indians try invade national congress b...        0
1   photographer equipment us $40 thousand stolen ...        0
2   the cynical opportunism marina silva. marina w...        0
3   fear discurses attacks "butcher republic": "th...        0
4   forensic psychiatrist concludes: "leftism ment...        0
5   disarming citizen good solution? brasil "3 mas...        0
6   mp/sp: "Thousands families injured family squi...        0
7   vatican police vatican flagra "gay orgy" couns...        0
8   after separation, nobrega andraceous says suff...        0
9   "profissa"! palocci says squid diverted money ...        0
10  public ministry give definite end pt, pmdb par...        0
11  japan athletes find toilets clogged Olympic vi...        0
12  air strike! minister defense contracted zika v...        0
13  pf clamp grava lull dilma proof appointment sa...        0
14  do think jô petista? trickled out! look porch ...  